# LightGBM & Optuna Example

In [1]:
%load_ext watermark
%watermark -p scikit-learn,optuna

scikit-learn: 1.0
optuna      : 2.10.0



- More info about Optuna: https://optuna.readthedocs.io/en/stable/

## Dataset

In [2]:
import pandas as pd


X_train = pd.read_csv('dataset/X_train.csv', header=None).values
y_train = pd.read_csv('dataset/y_train.csv', header=None).values.ravel().astype(int)

X_test = pd.read_csv('dataset/X_test.csv', header=None).values
y_test = pd.read_csv('dataset/y_test.csv', header=None).values.ravel().astype(int)

print('X_train.shape:', X_train.shape)
print('y_train.shape:', y_train.shape)
print('X_test.shape:', X_test.shape)
print('y_test.shape:', y_test.shape)

X_train.shape: (9119, 16)
y_train.shape: (9119,)
X_test.shape: (4492, 16)
y_test.shape: (4492,)


## Objective

In [3]:
import numpy as np
import optuna
from optuna.integration import LightGBMPruningCallback

import lightgbm

from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

import warnings

warnings.filterwarnings("ignore", category=UserWarning)
#optuna.logging.set_verbosity(optuna.logging.WARNING)


def objective(trial, X_train, y_train, cv=5):
    
    param_grid = {
        "n_estimators": trial.suggest_categorical("n_estimators", [10, 100]),
        "learning_rate": trial.suggest_categorical("learning_rate", [0.01]),
    }
    
    cv_iterator = StratifiedKFold(n_splits=cv, shuffle=True, random_state=123)

    cv_scores = np.zeros(cv)
    for idx, (train_sub_idx, valid_idx) in enumerate(cv_iterator.split(X_train, y_train)):
        
        X_train_sub, X_valid = X_train[train_sub_idx], X_train[valid_idx]
        y_train_sub, y_valid = y_train[train_sub_idx], y_train[valid_idx]

        model = lightgbm.LGBMClassifier(objective="multi_logloss", **param_grid)
        model.fit(
            X_train_sub,
            y_train_sub,
            eval_set=[(X_valid, y_valid)],
            eval_metric="multi_logloss",
            verbose=-1,
            early_stopping_rounds=50,
            callbacks=[
                LightGBMPruningCallback(trial=trial, metric="multi_logloss")
            ],  # Add a pruning callback to eliminate unpromising candidates
        )
        preds = model.score(X_valid, y_valid)
        
        cv_scores[idx] = preds

    return 1-np.mean(cv_scores)

In [4]:
study = optuna.create_study(direction="minimize", study_name="LGBM Classifier")

def func(trial):
    return objective(trial, X_train, y_train)

study.optimize(func, n_trials=50);

[I 2021-10-28 21:43:12,978] A new study created in memory with name: LGBM Classifier


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.456525
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.420827
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.417445
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.411489
Training until validation scores don't improve for 50 rounds


[I 2021-10-28 21:43:16,921] Trial 0 finished with value: 0.07478743828853529 and parameters: {'n_estimators': 100, 'learning_rate': 0.01}. Best is trial 0 with value: 0.07478743828853529.


Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.405919
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.456525
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.420827
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.417445
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.411489
Training until validation scores don't improve for 50 rounds


[I 2021-10-28 21:43:20,277] Trial 1 finished with value: 0.07478743828853529 and parameters: {'n_estimators': 100, 'learning_rate': 0.01}. Best is trial 0 with value: 0.07478743828853529.


...

Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.405919
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[10]	valid_0's multi_logloss: 1.44546
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[10]	valid_0's multi_logloss: 1.43974
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[10]	valid_0's multi_logloss: 1.43918
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[10]	valid_0's multi_logloss: 1.43894
Training until validation scores don't improve for 50 rounds


[I 2021-10-29 07:27:03,500] Trial 48 finished with value: 0.48612670939554037 and parameters: {'n_estimators': 10, 'learning_rate': 0.01}. Best is trial 0 with value: 0.07478743828853529.


Did not meet early stopping. Best iteration is:
[10]	valid_0's multi_logloss: 1.43555
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.456525
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.420827
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.417445
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.411489
Training until validation scores don't improve for 50 rounds


[I 2021-10-29 07:27:32,318] Trial 49 finished with value: 0.07478743828853529 and parameters: {'n_estimators': 100, 'learning_rate': 0.01}. Best is trial 0 with value: 0.07478743828853529.


Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.405919


In [5]:
print(f"\tBest value: {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

	Best value: 0.07479
	Best params:
		n_estimators: 100
		learning_rate: 0.01


In [6]:
model = lightgbm.LGBMClassifier(objective="multi_logloss", **study.best_params)
model.fit(X_train, y_train)

LGBMClassifier(learning_rate=0.01, objective='multi_logloss')

In [7]:
print(f"Training Accuracy: {model.score(X_train, y_train):0.2f}")
print(f"Test Accuracy: {model.score(X_test, y_test):0.2f}")

Training Accuracy: 0.95
Test Accuracy: 0.92
